<a href="https://colab.research.google.com/github/rachel1234561/DS2002_Project/blob/main/converting_and_summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rachel1234561/DS2002_Project.git

Cloning into 'DS2002_Project'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 45 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 4.61 MiB | 2.92 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
#we need to figure out 3. Modify the number of columns from the source to the destination,
#reducing or adding columns. If you add data cols you can put any other useful
#information in that vcolumn you wish and also converting from json.

In [2]:
import pandas as pd
import json
import requests
import sqlite3

In [40]:
#the function takes the data being converted and the format of the data
def convert_data(input_data, input_format):
    # make input data a pandas df and handle errors
    if input_format == 'csv':
        #save csv as df
        df = pd.read_csv(input_data)
        #print a summary of the data file ingestion
        num_records = len(df)
        num_columns = len(df.columns)
        print("Summary of the data file ingestion:")
        print("Number of records:" + str(num_records))
        print("Number of columns:" + str(num_columns))
        print("")
    elif input_format == 'json':
        #save json as df
        #convert to json first?
        #df = pd.DataFrame(input_data)
        data = json.loads(input_data)
        columns_data = data['meta']['view']['columns']
        columns = [column['name'] for column in columns_data]
        df = pd.DataFrame(data['data'], columns=columns)
        #df = pd.read_json(input_data)
        #print a summary of the data file ingestion
        print("summary of the data file ingestion:\n")
        print("Number of records:" + str(len(df)))
        print("Number of columns:" + str(df.columns))
    else:
        print("Invalid input format. Choose 'csv' or 'json'.")

    #ask use to input their prefered file type
    output_format = input('Enter output format (csv, json, sql): ')
    # Output data as the correct file type
    #saves a csv to files
    if output_format == 'csv':
        df.to_csv('output.csv')
        # Post-processing summary
        print("Post-processing summary:")
        print("Number of records:", len(pd.read_csv('output.csv')))
        print("Number of columns:", len(pd.read_csv('output.csv').columns))
    #saves a json file to files (also might need to debug this part)
    elif output_format == 'json':
        df.to_json('output.json')
        # Post-processing summary (I used stack overflow to debug this part)
        with open('output.json', 'r') as f:
         data = json.load(f)
        print("Post-processing summary:")
        print("Number of records:", len(df))
        print("Number of columns:", len(data))
    #saves to a sql db + db file in files
    elif output_format == 'sql':
      connection = sqlite3.connect('output.db')
      cursor = connection.cursor()
      df.to_sql('output', connection, if_exists='replace', index=False)
      connection.close()
      # Post-processing summary (SQL)
      connection = sqlite3.connect('output.db')
      cursor = connection.cursor()
      cursor.execute("SELECT COUNT(*) FROM output")
      num_records = cursor.fetchone()[0]
      cursor.execute("SELECT COUNT(*) FROM pragma_table_info('output')")
      num_columns = cursor.fetchone()[0]
      connection.close()
      print("Post-processing summary (SQL):")
      print("Number of records:", num_records)
      print("Number of columns:", num_columns)
    else:
        print("Invalid output format. Choose 'csv', 'json', or 'sql'.")

In [5]:
#pass the path to the csv to the function
#this overall seems to be working
#csv file path (from cloning the project repo) --> '/content/DS2002_Project/cereal.csv'
convert_data('/content/DS2002_Project/cereal.csv', 'csv')

Summary of the data file ingestion:
Number of records:77
Number of columns:16

Enter output format (csv, json, sql): sql
Post-processing summary (SQL):
Number of records: 77
Number of columns: 16


In [6]:
#this is to test to see if the SQL part is working
connection = sqlite3.connect('output.db')
cursor = connection.cursor()
cursor.execute("SELECT * FROM output")
rows = cursor.fetchall()
for row in rows:
    print(row)
connection.close()

('100% Bran', 'N', 'C', 70, 4, 1, 130, 10.0, 5.0, 6, 280, 25, 3, 1.0, 0.33, 68.402973)
('100% Natural Bran', 'Q', 'C', 120, 3, 5, 15, 2.0, 8.0, 8, 135, 0, 3, 1.0, 1.0, 33.983679)
('All-Bran', 'K', 'C', 70, 4, 1, 260, 9.0, 7.0, 5, 320, 25, 3, 1.0, 0.33, 59.425505)
('All-Bran with Extra Fiber', 'K', 'C', 50, 4, 0, 140, 14.0, 8.0, 0, 330, 25, 3, 1.0, 0.5, 93.704912)
('Almond Delight', 'R', 'C', 110, 2, 2, 200, 1.0, 14.0, 8, -1, 25, 3, 1.0, 0.75, 34.384843)
('Apple Cinnamon Cheerios', 'G', 'C', 110, 2, 2, 180, 1.5, 10.5, 10, 70, 25, 1, 1.0, 0.75, 29.509541)
('Apple Jacks', 'K', 'C', 110, 2, 0, 125, 1.0, 11.0, 14, 30, 25, 2, 1.0, 1.0, 33.174094)
('Basic 4', 'G', 'C', 130, 3, 2, 210, 2.0, 18.0, 8, 100, 25, 3, 1.33, 0.75, 37.038562)
('Bran Chex', 'R', 'C', 90, 2, 1, 200, 4.0, 15.0, 6, 125, 25, 1, 1.0, 0.67, 49.120253)
('Bran Flakes', 'P', 'C', 90, 3, 0, 210, 5.0, 13.0, 5, 190, 25, 3, 1.0, 0.67, 53.313813)
("Cap'n'Crunch", 'Q', 'C', 120, 1, 2, 220, 0.0, 12.0, 12, 35, 25, 2, 1.0, 0.75, 18.04285

In [11]:
#I used stack to figure out how to show the output.json in pretty print to make sure the function works
with open('output.json', 'r') as f:
  data = json.load(f)
print(json.dumps(data, indent=1))
#ask if this is ok

{
 "name": {
  "0": "100% Bran",
  "1": "100% Natural Bran",
  "2": "All-Bran",
  "3": "All-Bran with Extra Fiber",
  "4": "Almond Delight",
  "5": "Apple Cinnamon Cheerios",
  "6": "Apple Jacks",
  "7": "Basic 4",
  "8": "Bran Chex",
  "9": "Bran Flakes",
  "10": "Cap'n'Crunch",
  "11": "Cheerios",
  "12": "Cinnamon Toast Crunch",
  "13": "Clusters",
  "14": "Cocoa Puffs",
  "15": "Corn Chex",
  "16": "Corn Flakes",
  "17": "Corn Pops",
  "18": "Count Chocula",
  "19": "Cracklin' Oat Bran",
  "20": "Cream of Wheat (Quick)",
  "21": "Crispix",
  "22": "Crispy Wheat & Raisins",
  "23": "Double Chex",
  "24": "Froot Loops",
  "25": "Frosted Flakes",
  "26": "Frosted Mini-Wheats",
  "27": "Fruit & Fibre Dates; Walnuts; and Oats",
  "28": "Fruitful Bran",
  "29": "Fruity Pebbles",
  "30": "Golden Crisp",
  "31": "Golden Grahams",
  "32": "Grape Nuts Flakes",
  "33": "Grape-Nuts",
  "34": "Great Grains Pecan",
  "35": "Honey Graham Ohs",
  "36": "Honey Nut Cheerios",
  "37": "Honey-comb",
 

In [43]:
#we might need to find a new api to start with (or fix the code) because it is mostly not working for this part
#the data is confusing (it shows by runing the next chunk of code)
url = 'https://data.wa.gov/api/views/f6w7-q2d2/rows.json?accessType=DOWNLOAD'
response = requests.get(url)
convert_data(response.text, 'json')

summary of the data file ingestion:

Number of records:210165
Number of columns:Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'VIN (1-10)', 'County', 'City', 'State',
       'Postal Code', 'Model Year', 'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract', 'Counties',
       'Congressional Districts',
       'WAOFM - GIS - Legislative District Boundary'],
      dtype='object')
Enter output format (csv, json, sql): json
Post-processing summary:
Number of records: 210165
Number of columns: 28
